# Compare-Algorithm

In [401]:
import pandas as pd
""" 
Read in two csv files and store in DataFrames. Merge DataFrames to get get correctnes of 
total-, in- and out_direction of bees regarding their in_videos and out_videos  
"""

# delete existing result.csv file to prevent errors
if (os.path.isfile('./result.csv')):
    os.remove("result.csv")

# store data in dataFrame
gtd = pd.read_csv('GroundTruthData.csv')
mld = pd.read_csv('MaschineLearnedData.csv')

# delete dublications
print('Delete dublication from input.')
mld = mld.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')
gtd = gtd.drop_duplicates(['bee_id', 'time_in', 'time_out'], keep='first')

# change DT from float to int to have the same DT like gtd
mld.bee_id = mld.bee_id.astype('int32')

# merge to get 100% correct matches
mergeTotalCorrect = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out', 'in_direction', 'out_direction'], how='inner')
#merge to get in_direction correct
mergeInDirection = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'in_direction'], how='inner')
#merge to get out_direction correct
mergeOutDirection = pd.merge(gtd, mld, on=['bee_id', 'time_out', 'out_direction'], how='inner')
# merge to get comparable bees to get all found bees
mergeableBees = pd.merge(gtd, mld, on=['bee_id', 'time_in', 'time_out'], how='inner')

# iterate mergeableBees for visualization
result = pd.DataFrame()
result['bee_id'] = mergeableBees['bee_id'] 
result['in_equals'] = mergeableBees['in_direction_x'] == mergeableBees['in_direction_y']
result['out_equals'] = mergeableBees['out_direction_x'] == mergeableBees['out_direction_y']

# store to result.csv
result.to_csv('result.csv', encoding='utf-8', index=False)

# calculate statitics
inCorrect = round(result['in_equals'].sum()/len(result) * 100)
outCorrect = round(result['out_equals'].sum()/len(result) * 100)
allCorrect = round((result['in_equals'] == result['out_equals']).sum()/len(result) * 100)

# print statistics
try:
    print(len(gtd), 'entries in GroundTruthData.', len(mergeableBees), 'entries found to compare in MaschineLearnedData.' )
    print(allCorrect, "% total correctness")
    print(inCorrect, "% Ingoing correctness")
    print(outCorrect, "% Outgoing correctness")
    print(round(((allCorrect + inCorrect + outCorrect) / 3), 2), "% Average correctness")
except ZeroDivisionError:
    print("Nothing to compare!")

# end compare algorithm
print('\nPlease check result.csv file or hit \'result\' for more details.')
print('Check also DataFrames for deeper information: \'mergeTotalCorrect\',\'mergeInCorrect\',\'mergeOutCorrect\',\'mergeableBees\' ')


Delete dublication from input.
103 entries in GroundTruthData. 44 entries found to compare in MaschineLearnedData.
70.0 % total correctness
77.0 % Ingoing correctness
89.0 % Outgoing correctness
78.67 % Average correctness

Please check result.csv file or hit 'result' for more details.
Check also DataFrames for deeper information: 'mergeTotalCorrect','mergeInCorrect','mergeOutCorrect','mergeableBees' 
